<a href="https://colab.research.google.com/github/MuskanBansal2001/MuskanBansal2001/blob/main/Tagging_of_Products.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Check the gpu
P100 > T4 > P4 > K80

In [ ]:
from IPython.display import HTML, clear_output
from subprocess import getoutput
s = getoutput('nvidia-smi')
if 'K80' in s:gpu = 'K80'
elif 'T4' in s:gpu = 'T4'
elif 'P100' in s:gpu = 'P100'
elif 'P4' in s:gpu = 'P4'
display(HTML(f"<h1>{gpu}</h1>"))

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/MyDrive/experiment2/

Mounted at /gdrive
/gdrive/MyDrive/experiment2


In [ ]:
!pip install -q yolov4==2.1.0

In [ ]:
from os.path import join, exists
from os import listdir
import json


from global_vars import *
from helpers import load_image_from_link, load_image_from_path
from model import load_normal_models, load_2ndlvl_models
from inference_helpers import *

from primary_image_detector import *

Call tf.config.experimental.set_memory_growth(GPU0, True)


## Load Models

In [ ]:
posenet_model = load_posenet_model()

topwear_models=load_normal_models(topwear)

outerwear_models=load_normal_models(outerwear)
outerwear_models["sub"]["edge_type"]=topwear_models["sub"]["edge_type"]

bottomwear_models=load_normal_models(bottomwear)

footwear_models=load_normal_models(footwear)

overall_models=load_normal_models(overall)

accessories_models=load_normal_models(accessories)

In [ ]:
topwear_nonhuman = "topwear_nonhuman"
topwear_nonhuman_model = load_normal_models(topwear_nonhuman)

bottomwear_nonhuman = "bottomwear_nonhuman"
bottomwear_nonhuman_model = load_normal_models(bottomwear_nonhuman)

In [ ]:
# img = load_image_from_link("https://i.ebayimg.com/images/g/xRYAAOSw~oFXGnNz/s-l400.jpg")
from primary_image_detector import predict_pose
def hasHuman(img, posenet_model):
    lst, image, disp = predict_pose(img, posenet_model)
    lst = sorted(lst, key = lambda x:x[2], reverse=True)
    if (lst[0][2]>0.5) and (lst[1][2]>0.5):
        # print(lst)
        return True
    else:
        # print(lst)
        return False
# hasHuman(img, posenet_model)

In [ ]:
from PIL.Image import fromarray as fa

In [ ]:
topwear_main_model_conf = 0.04
topwear_nonhuman_main_model_conf = 0.04
bottomwear_main_model_conf = 0.2
bottomwear_nonhuman_main_model_conf = 0.2
accessories_main_model_conf = 0.2
footwear_main_model_conf = 0.2

In [ ]:
def predict(url, found = None):

    # Image downloading and finding primary image if required
    if type(url) == list:
        img_list = []
        for i in url:
            if type(i)!=str:
                raise Exception("Urls list should contain only string type elements.")
            img_list.append(load_image_from_link(i))
        ind = find_primary(img_list, posenet_model)
        image_link = url[ind]
        img = img_list[ind]
    elif type(url) == str:
        image_link = url
        img = load_image_from_link(url)
    else:
        raise Exception("Url parameter should be string type or a list of string type.")
    #check if it has human or not
    # hash = hasHuman(img, posenet_model)
    # print(hash)
    hash = True

    # initialising all the variables
    topwear_detection = []
    bottomwear_detection = []
    accessories_sub_detection = {}
    footwear_detection = []

    bottomwear_sub_detection={}
    topwear_sub_detection={}
    overall_sub_detection={}
    outerwear_sub_detection={}

    # resolving the already known thing
    if not hash:
        if found == None:
            topwear_detection, topwear_color = topwear_nonhuman_model["main"].predict(img, 0.1, topwear_nonhuman_main_model_conf)
            bottomwear_detection, bottomwear_color = bottomwear_nonhuman_model["main"].predict(img, 0.1, bottomwear_nonhuman_main_model_conf)
            accessories_sub_detection = accessories_prediction(img, accessories_models['main'], ren, 0.1, accessories_main_model_conf)
            footwear_detection, footwear_color = footwear_models["main"].predict(img, 0.1, footwear_main_model_conf)
        elif found in topwear_models["main"].class_dict.values():
            topwear_detection = [[found, 1]]
        elif found == topwear:
            topwear_detection, topwear_color = topwear_nonhuman_model["main"].predict(img, 0.1, topwear_nonhuman_main_model_conf)
        elif found in bottomwear_models["main"].class_dict.values():
            bottomwear_detection = [[found, 1]]
        elif found == bottomwear:
            bottomwear_detection, bottomwear_color = bottomwear_nonhuman_model["main"].predict(img, 0.1, bottomwear_nonhuman_main_model_conf)
        elif (found == accessories) or (found in accessories_models["main"].class_dict.values()):
            accessories_sub_detection = accessories_prediction(img, accessories_models['main'], ren, 0.1, accessories_main_model_conf)
        elif (found == footwear) or (found in footwear_models["main"].class_dict.values()):
            footwear_detection, footwear_color = footwear_models["main"].predict(img, 0.1, footwear_main_model_conf)
        else:
            raise Exception(found+" not found in any model.")
    else:
        if found == None:
            topwear_detection, topwear_color = topwear_models["main"].predict(img, 0.1, topwear_main_model_conf)
            bottomwear_detection, bottomwear_color = bottomwear_models["main"].predict(img, 0.1, bottomwear_main_model_conf)
            accessories_sub_detection = accessories_prediction(img, accessories_models['main'], ren, 0.1, accessories_main_model_conf)
            footwear_detection, footwear_color = footwear_models["main"].predict(img, 0.1, footwear_main_model_conf)
        elif found in topwear_models["main"].class_dict.values():
            topwear_detection = [[found, 1]]
        elif found == topwear:
            topwear_detection, topwear_color = topwear_models["main"].predict(img, 0.1, topwear_main_model_conf)
        elif found in bottomwear_models["main"].class_dict.values():
            bottomwear_detection = [[found, 1]]
        elif found == bottomwear:
            bottomwear_detection, bottomwear_color = bottomwear_models["main"].predict(img, 0.1, bottomwear_main_model_conf)
        elif (found == accessories) or (found in accessories_models["main"].class_dict.values()):
            accessories_sub_detection = accessories_prediction(img, accessories_models['main'], ren, 0.1, accessories_main_model_conf)
        elif (found == footwear) or (found in footwear_models["main"].class_dict.values()):
            footwear_detection, footwear_color = footwear_models["main"].predict(img, 0.1, footwear_main_model_conf)
        else:
            raise Exception(found+" not found in any model.")

    # making further predictions
    if len(topwear_detection)!=0:
        topwear_detection = [topwear_detection[0]]
        if topwear_detection[0][0] in ['bodysuit', 'jumpsuit', 'kaftan', 'dress']:
            if (len(bottomwear_detection)==0) or (topwear_detection[0][1]>bottomwear_detection[0][1]):
                overall_sub_detection = sub_model_prediction(img, topwear_detection, overall_models['sub'], 0.1, 0.1)
            else:topwear_detection = []
        elif topwear_detection[0][0] in ['blazers', 'cardigan', 'coat', 'jacket']:
            outerwear_sub_detection = sub_model_prediction(img, topwear_detection, outerwear_models['sub'], 0.1, 0.1)
        else:
            topwear_sub_detection = sub_model_prediction(img, topwear_detection, topwear_models['sub'], 0.1, 0.1)

    bottomwear_sub_detection = sub_model_prediction(img, bottomwear_detection, bottomwear_models['sub'], 0.1, 0.1)

    footwear_sub_detection = sub_model_prediction(img, footwear_detection, footwear_models['sub'], 0.1, 0.1)
    if len(footwear_detection)!=0:
        footwear_sub_detection['color']=transform_color(footwear_color)

    # transforming results
    trans_result = transform_result({
                    topwear:topwear_sub_detection,
                    bottomwear:bottomwear_sub_detection,
                    overall:overall_sub_detection,
                    outerwear:outerwear_sub_detection,
                    footwear:footwear_sub_detection,
                    accessories:accessories_sub_detection
                },ren)
    trans_result["found_primary"] = image_link
    return trans_result

In [ ]:
f=open("rename.json", "r")
ren = json.load(f)
f.close()

In [ ]:
# url = "https://images.unsplash.com/photo-1563389234808-52344934935c?ixid=MnwxMjA3fDB8MHxzZWFyY2h8OXx8c2hpcnR8ZW58MHx8MHx8&ixlib=rb-1.2.1&w=1000&q=80"
# # url = "https://imagesvc.meredithcorp.io/v3/mm/image?url=https%3A%2F%2Fstatic.onecms.io%2Fwp-content%2Fuploads%2Fsites%2F14%2F2019%2F04%2F09%2F040919-sexy-cardigan-lead-2000.jpg&q=85"
# # url = "https://cdn.shopify.com/s/files/1/0915/5382/products/KK2447_Casey_Longline_Cardigan_Black_01_1024x1024@3x.jpg"
# print(json.dumps(predict(url), indent=4))

In [ ]:
import json
from multiprocess import Pool as ThreadPool
import requests
import time
import sys

In [ ]:
f = open("grroomTagging/productRename_2.json", "r")
productRename = json.load(f)
f.close()
dont_need_to_run = []
need_to_run = []
for i,j in productRename.items():
    if j==None:
        dont_need_to_run.append(i)
    else:
        need_to_run.append(i)

In [ ]:
def prep_payload(d):
    present = productRename[d["productCategory"]]
    if present in ["topwear","blazers","bodysuit","brasserie","cardigan","coat","dress","jacket","jumpsuit","kaftan","poncho","shirts","sweatshirt","tops","bottomwear","culottes","jeans","leggings","shorts","skirt","sweatpant","trouser","underpants"]:
        url = d["secondaryImageLinks"]
        url.append(d['primaryImageLink'])
    else:
        url = d["primaryImageLink"]
    return [url, present]

In [ ]:
target_file = input("Enter name of the file (Ex: Myntra_Iteration_1_1.json):\n")
if target_file in listdir("grroomTagging/raw"):
    f = open("grroomTagging/raw/"+target_file, "r")
    tmp = json.load(f)
    f.close()
    if target_file in listdir("grroomTagging/processed"):
        resp = input("Part of the file is already processed, want to resume (y/n)")
        if resp=="n":pass
        elif resp =="y":
            f = open("grroomTagging/processed/"+target_file, "r")
            proc = json.load(f)
            proc_id = [i["_id"] for i in proc]
            f.close()
            raw_data = []
            for i in tmp:
                if i["_id"] not in proc_id:
                    raw_data.append(i)
            print("Total products selected:", len(raw_data))
        else:
            print("Response should be either 'y' or 'n'!")
    else:
        raw_data = tmp
        proc = []
        print("Total products selected:", len(raw_data))
else:
    print("Some error in the file name, please correct it.")


Enter name of the file (Ex: Myntra_Iteration_1_1.json):
Metroshoes_Iteration_2_1.json
Total products selected: 74


In [ ]:
total_products = len(raw_data)
start = time.time()
gender_dict = {
    "Men":"Male",
    "Women":"Female",
    "Unisex":"Others"
}
for i in range(total_products):
    avgTime = round((time.time()-start)/(i+1), 1)
    etc = round(avgTime*(total_products-i)/60, 1)
    te = round((time.time()-start)/60, 1)
    status = "\n{}/{}, AvgTime:{}, ETC {} mints, TimeElapsed: {} mints".format(i, total_products, avgTime, etc, te)
    sys.stdout.write(status)
    url, present = prep_payload(raw_data[i])
    try:
        data = predict(url, present)
        data = {**data, **raw_data[i]}
        data["gender"] = gender_dict[data["gender"]]
        proc.append(data)
        if len(proc)%100==0:
            f=open("grroomTagging/processed/"+target_file, "w")
            json.dump(proc, f)
            f.close()
        sys.stdout.write("\b" * (len(status)))
    except KeyboardInterrupt:
        break
    except:
      print(url)
      continue


0/74, AvgTime:0.0, ETC 0.0 mints, TimeElapsed: 0.0 mintsWARNING:tensorflow:5 out of the last 5 calls to <function Model.make_predict_function.<locals>.predict_function at 0x7f020e656440> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.

1/74, AvgTime:4.8, ETC 5.8 mints, TimeElapsed: 0.2 mintsWARNING:tensorflow:6 out of the last 7 calls to <function Model.make_predict_function.<locals